# extract table data

In [15]:
import pandas as pd
import numpy as np
from selenium import webdriver

In [48]:
browser = webdriver.Chrome("chromedriver.exe")

In [49]:
url = "https://www.imdb.com/india/top-rated-indian-movies/?ref_=nv_mv_250_in"
browser.get(url)

In [50]:
table = pd.read_html(url)
len(table)

2

In [23]:
movies = table[0]
movies.head()

,Unnamed: 0,Rank & Title,IMDb Rating,Your Rating,Unnamed: 4
0,NaN,1. Anand (1971),8.7,12345678910 NOT YET RELEASED Seen,NaN
1,NaN,2. Drishyam (2013),8.6,12345678910 NOT YET RELEASED Seen,NaN
2,NaN,3. Nayakan (1987),8.6,12345678910 NOT YET RELEASED Seen,NaN
3,NaN,4. Anbe Sivam (2003),8.5,12345678910 NOT YET RELEASED Seen,NaN
4,NaN,5. Gol Maal (1979),8.5,12345678910 NOT YET RELEASED Seen,NaN


In [25]:
#movies.drop(['Unnamed: 0','Unnamed: 4'],axis=1,inplace=True)#removing first and last columns

In [26]:
movies.head()

,Rank & Title,IMDb Rating,Your Rating
0,1. Anand (1971),8.7,12345678910 NOT YET RELEASED Seen
1,2. Drishyam (2013),8.6,12345678910 NOT YET RELEASED Seen
2,3. Nayakan (1987),8.6,12345678910 NOT YET RELEASED Seen
3,4. Anbe Sivam (2003),8.5,12345678910 NOT YET RELEASED Seen
4,5. Gol Maal (1979),8.5,12345678910 NOT YET RELEASED Seen


# Method 2:using Selenium

In [51]:
#other ways to open the data
table_tag = browser.find_element_by_css_selector('table.full-width[data-caller-name="imdb-featured-india"]')
table_tag

<selenium.webdriver.remote.webelement.WebElement (session="bdfe7232bc383bd2e916f7980c6260ef", element="0.8774644237397615-1")>

In [52]:
thead = table_tag.find_element_by_css_selector("thead")
tr = thead.find_element_by_css_selector("tr")
th_tags = tr.find_elements_by_css_selector("th")
len(th_tags)

5

In [53]:
col_names = [th.text for th in th_tags]
col_names

['', 'Rank & Title', 'IMDb Rating', 'Your Rating', '']

In [30]:
#th_tag = table_tag.find_elements_by_css_selector("thead>tr>th")
#len(th_tag)

5

In [ ]:
#thead-for columns
#tbody contains text
#within tbody there is tr tag which represent a row
#th is table column

# Getting Data

In [55]:

col_names[0]="Poster"
col_names[-1]="Watchlist"
imdb = pd.DataFrame(columns=col_names)
tr_tags = table_tag.find_elements_by_css_selector("tbody>tr")#total 250 rows are there
len(tr_tags)

250

In [44]:

for tr in tr_tags:
    td_tags = tr.find_elements_by_css_selector("td")
    td_data = [td.text for td in td_tags]
    row_series = pd.Series(td_data,index=col_names)
    imdb = imdb.append(row_series,ignore_index=True)
    

In [45]:
imdb.head()

,Poster,Rank & Title,IMDb Rating,Your Rating,Watchlist
0,,1. Anand (1971),8.7,,
1,,2. Drishyam (2013),8.6,,
2,,3. Nayakan (1987),8.6,,
3,,4. Anbe Sivam (2003),8.5,,
4,,5. Gol Maal (1979),8.5,,


In [41]:
#`imdb.drop(["Your Rating","Watchlist"],axis=1,inplace=True)

In [42]:
imdb.head()

,Poster,Rank & Title,IMDb Rating
0,,1. Anand (1971),8.7
1,,2. Drishyam (2013),8.6
2,,1. Anand (1971),8.7
3,,2. Drishyam (2013),8.6
4,,3. Nayakan (1987),8.6


In [56]:
for tr in tr_tags:
    td_tags = tr.find_elements_by_css_selector("td")
    td_data = [td.text for td in td_tags]
    td_img_first = td_tags[0].find_element_by_css_selector("img").get_attribute("src")#getting the image url
    td_data[0] = td_img_first
    row_series = pd.Series(td_data,index=col_names)
    imdb = imdb.append(row_series,ignore_index=True)
    

In [57]:
imdb.head()

,Poster,Rank & Title,IMDb Rating,Your Rating,Watchlist
0,https://m.media-amazon.com/images/M/MV5BNmZkMT...,1. Anand (1971),8.7,,
1,https://m.media-amazon.com/images/M/MV5BYmY3Mz...,2. Drishyam (2013),8.6,,
2,https://m.media-amazon.com/images/M/MV5BZDhkMT...,3. Nayakan (1987),8.6,,
3,https://m.media-amazon.com/images/M/MV5BNWQxY2...,4. Anbe Sivam (2003),8.5,,
4,https://m.media-amazon.com/images/M/MV5BMjA4OT...,5. Gol Maal (1979),8.5,,


In [64]:
#download all images link\
import urllib
img_link = imdb["Poster"][0]
img_title = imdb["Rank & Title"][0]


In [67]:
#x[x.rfind]
print(img_link, img_title)
urllib.request.urlretrieve(img_link,img_title)

https://m.media-amazon.com/images/M/MV5BNmZkMTMzNmEtMWU5NC00MjEzLWE5MzktYzRlMmQyMzk0YmM1XkEyXkFqcGdeQXVyNTA4NzY1MzY@._V1_UX45_CR0,0,45,67_AL_.jpg 1. Anand (1971)


('1. Anand (1971)', <http.client.HTTPMessage at 0x1e940f17a90>)

In [68]:
img_extn = img_link[img_link.rfind("."):]
img_extn

'.jpg'

In [69]:
img_name = img_title+img_extn
img_name

'1. Anand (1971).jpg'

In [74]:
import urllib
img_link = imdb["Poster"][0]
img_title = imdb["Rank & Title"][0]
img_extn = img_link[img_link.rfind("."):]
img_name = img_title+img_extn

print(img_link, img_name)
urllib.request.urlretrieve(img_link,img_name)

https://m.media-amazon.com/images/M/MV5BNmZkMTMzNmEtMWU5NC00MjEzLWE5MzktYzRlMmQyMzk0YmM1XkEyXkFqcGdeQXVyNTA4NzY1MzY@._V1_UX45_CR0,0,45,67_AL_.jpg 1. Anand (1971).jpg


('1. Anand (1971).jpg', <http.client.HTTPMessage at 0x1e940f08358>)

In [78]:
import os
import glob

In [ ]:
#for index,row in imdb.iterrows():
#    print(row["Rank & Title"],row["Poster"])

    img_link = imdb["Poster"][0]
    img_title = imdb["Rank & Title"][0]
    img_extn = img_link[img_link.rfind("."):]#find last extension is .jpeg
    
    folder_name = "movies_name"
    if not folder_name in glob.glob("*"):
        os.makedirs(folder_name)
    img_name = folder_name+"/"+img_title+img_extn

    print(img_link, img_name)
    urllib.request.urlretrieve(img_link,img_name)

In [79]:
import os
import glob

folder_name = "movies_name"
if not folder_name in glob.glob("*"):
    os.makedirs(folder_name)
for index,row in imdb.iterrows():
    img_link = row["Poster"]
    img_title = row["Rank & Title"]
    img_extn = img_link[img_link.rfind("."):]
    img_name = folder_name+"/"+img_title+img_extn
    urllib.request.urlretrieve(img_link,img_name)

In [81]:
glob.glob("*")

['1. Anand (1971)',
 '1. Anand (1971).jpg',
 'books.html',
 'chromedriver.exe',
 'Class-1(05-03-2019).ipynb',
 'Class-2(06-03-2019).ipynb',
 'Class-3(11-03-2019).ipynb',
 'debug.log',
 'index.html',
 'index1.html',
 'jobs_datascience.csv',
 'movies_name',
 'new_book.html',
 'OYO.ipynb',
 'treebo.ipynb',
 'untitled.txt']

In [84]:
#imdb.to_csv("name for your csv",index=False)
glob.glob("movies_name/*")

['movies_name\\1. Anand (1971).jpg',
 'movies_name\\10. Andhadhun (2018).jpg',
 'movies_name\\100. Manjhi',
 'movies_name\\101. Ankhon Dekhi (2013).jpg',
 'movies_name\\102. Pizza (2012).jpg',
 'movies_name\\103. Jab We Met (2007).jpg',
 'movies_name\\104. K.G.F',
 'movies_name\\105. Padman (2018).jpg',
 'movies_name\\106. Airlift (2016).jpg',
 'movies_name\\107. Kal Ho Naa Ho (2003).jpg',
 'movies_name\\108. Hey Ram (2000).jpg',
 'movies_name\\109. Baby (2015).jpg',
 'movies_name\\11. Dangal (2016).jpg',
 'movies_name\\110. Rangasthalam (2018).jpg',
 'movies_name\\111. Vaaranam Aayiram (2008).jpg',
 'movies_name\\112. My Name Is Khan (2010).jpg',
 'movies_name\\113. Secret Superstar (2017).jpg',
 'movies_name\\114. Lakshya (2004).jpg',
 'movies_name\\115. Baahubali',
 'movies_name\\116. Border (1997).jpg',
 'movies_name\\117. Ab Tak Chhappan (2004).jpg',
 'movies_name\\118. Gangaajal (2003).jpg',
 'movies_name\\119. Vettaiyaadu Vilaiyaadu (2006).jpg',
 'movies_name\\12. Pather Panchal